<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Modeling_ICG_svr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
!pip install catboost > log.txt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
import random
torch.manual_seed(11)
np.random.seed(11)
random.seed(11)

In [53]:
def get_col(df,name):
  return df.columns[df.columns.str.contains(name)]

![img](https://i.imgur.com/hksoTrh.jpg)



# load data

In [54]:
c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c.keys(),c['x'],c['y'],len(c['x']),len(c['y'])

(dict_keys(['x', 'y']),
 ['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    Toluene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    Toluene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from T651    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651    Toluene_wt%',
  'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%',
  'Simulation Case Conditions_Spec 2 : NA in Ben

In [55]:
df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
df.head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
001-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,980.0,0.5
001-005,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,950.0,1.0
001-008,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,920.0,1.5
001-011,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,980.0,0.5
001-014,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,950.0,1.0


# Distillate Rate_m3/hr == 0.01的部份 改成 0

In [56]:
df.loc[df['Simulation Case Conditions_C620 Distillate Rate_m3/hr']==0.01,'Simulation Case Conditions_C620 Distillate Rate_m3/hr'] = 0

In [57]:
df[c['y']].max(),df[c['y']].min()

(Simulation Case Conditions_C620 Distillate Rate_m3/hr    10.25
 dtype: float64, Simulation Case Conditions_C620 Distillate Rate_m3/hr    0.0
 dtype: float64)

# modeling 不要限制輸出 可以有負值

In [58]:
import catboost
from sklearn.model_selection import train_test_split
X = df[c['x']]
y = df[c['y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [59]:
c620_icg = Pipeline([
                     ('scaler',StandardScaler()),
                     ('lr',SVR())])
c620_icg.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lr',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

# evaluate

In [60]:
from sklearn.metrics import r2_score,mean_squared_error

def mape(y_true, y_pred, e = 1e-8):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true > e
    y_true, y_pred = y_true[mask], y_pred[mask]
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def show_metrics(y_real,y_pred,e=1e-8):
    res = pd.DataFrame(index=y_pred.columns,columns=['R2','MSE','MAPE'])
    for i in y_pred.columns:
        res.loc[i,'R2'] = np.clip(r2_score(y_real[i],y_pred[i]),0,1)
        res.loc[i,'MSE'] = mean_squared_error(y_real[i],y_pred[i])
        res.loc[i,'MAPE'] = mape(y_real[i],y_pred[i],e)
    res.loc['AVG'] = res.mean(axis=0)
    return res

In [61]:
y_real = y_test
y_pred = pd.DataFrame(c620_icg.predict(X_test),index=y_real.index,columns=y_real.columns)
show_metrics(y_real,y_pred)

,R2,MSE,MAPE
Simulation Case Conditions_C620 Distillate Rate_m3/hr,0.896758,0.185515,34.1956
AVG,0.896758,0.185515,34.1956


# ICG while 迴圈

In [62]:
def ICG(Input):
  while True:
    output = pd.DataFrame(c620_icg.predict(Input.values),index=Input.index,columns=['Simulation Case Conditions_C620 Distillate Rate_m3/hr'])
    dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
    na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    print('current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{}'.format(dist_rate,na_in_benzene))
    if output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0] > 0:
      return output,Input
    else:
      Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
      print('NA in Benzene_ppmw -= 30')

In [63]:
print(df.shape)
df = df[df['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']==70.0]
df = df[df['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']==980.0]
print(df.shape)

(1296, 15)
(122, 15)


In [64]:
for i in range(10):
  Input = df[c['x']].sample(1)
  output,Input = ICG(Input)
  print(i)

current Distillate Rate_m3/hr:-0.10047456630132845 NA in Benzene_ppmw:980.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.026534339265856355 NA in Benzene_ppmw:950.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:0.07779439557231149 NA in Benzene_ppmw:920.0
0
current Distillate Rate_m3/hr:3.4237286889824405 NA in Benzene_ppmw:980.0
1
current Distillate Rate_m3/hr:-0.07194247226384953 NA in Benzene_ppmw:980.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.008595871250028697 NA in Benzene_ppmw:950.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:0.09967207799939004 NA in Benzene_ppmw:920.0
2
current Distillate Rate_m3/hr:0.544147088443449 NA in Benzene_ppmw:980.0
3
current Distillate Rate_m3/hr:0.49899747805330663 NA in Benzene_ppmw:980.0
4
current Distillate Rate_m3/hr:0.03759480054163955 NA in Benzene_ppmw:980.0
5
current Distillate Rate_m3/hr:4.099537896009487 NA in Benzene_ppmw:980.0
6
current Distillate Rate_m3/hr:1.0090189564328484 NA in Benze

# 特別資料測試

In [65]:
Input = {
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr': {0: 183.71834522298502},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%': {0: 2.06},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%': {0: 11.91},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%': {0: 37.48},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr': {0: 18.599028347620738},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%': {0: 2.51},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%': {0: 69.02},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%': {0: 20.62},
'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr': {0: 103.76780384010259},
'Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%': {0: 0.685},
'Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%': {0: 46.628},
'Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%': {0: 40.868},
'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%': {0: 89.39},
'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': {0: 849.0}}
Input = pd.DataFrame(Input)
output,Input = ICG(Input)

current Distillate Rate_m3/hr:1.0426470642695531 NA in Benzene_ppmw:849.0


In [66]:
Input = {
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr': {0: 171.58942010720452},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%': {0: 1.971},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%': {0: 12.146},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%': {0: 37.791},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr': {0: 19.669237218735567},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%': {0: 2.56},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%': {0: 68.421},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%': {0: 21.33},
'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr': {0: 101.95451869681709},
'Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%': {0: 0.777},
'Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%': {0: 50.531},
'Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%': {0: 35.435},
'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%': {0: 89.851},
'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': {0: 1193.0}}
Input = pd.DataFrame(Input)
output,Input = ICG(Input)

current Distillate Rate_m3/hr:1.0426470642695531 NA in Benzene_ppmw:1193.0


In [67]:
df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
Input = df.loc['024-008',c['x']].to_frame().T
Input

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
024-008,200.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.4524,44.244236,44.786339,90.0,800.0


In [68]:
Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 920
Input

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
024-008,200.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.4524,44.244236,44.786339,90.0,920


In [69]:
c620_icg.predict(Input.values)

array([0.15557103])

In [70]:
_,_ = ICG(Input)

current Distillate Rate_m3/hr:0.1555710276149881 NA in Benzene_ppmw:920


In [71]:
joblib.dump(c620_icg,'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_svr.pkl')

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_svr.pkl']